<a href="https://colab.research.google.com/github/RS-Sowmya/Learning/blob/main/kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
from google .colab import files
upload1 = files.upload()
upload2 = files.upload()

Saving kaggle_train.csv to kaggle_train (4).csv


Saving kaggle_test.csv to kaggle_test (2).csv


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [67]:
df = pd.read_csv("kaggle_train.csv")
df_test = pd.read_csv("kaggle_test.csv")

In [68]:
df['ExtendedQuantity'] = pd.to_numeric(df['ExtendedQuantity'], errors='coerce')
df['QtyShipped'] = pd.to_numeric(df['QtyShipped'], errors='coerce')
num_col = ['MW','SIZE_BUILDINGSIZE', 'NUMFLOORS', 'NUMROOMS','NUMBEDS','invoiceTotal', 'UnitPrice', 'ExtendedPrice','QtyShipped', 'ExtendedQuantity']
cat_col = ['UOM','PriceUOM']
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy="median")
df[num_col] = num_imputer.fit_transform(df[num_col])
cat_imputer = SimpleImputer(strategy="most_frequent")
df[cat_col] = cat_imputer.fit_transform(df[cat_col])

In [69]:
df.isnull().sum()

,0
id,0
PROJECTNUMBER,0
MW,0
PROJECT_CITY,0
STATE,0
PROJECT_COUNTRY,0
CORE_MARKET,0
PROJECT_TYPE,2
SIZE_BUILDINGSIZE,0
NUMFLOORS,0


In [70]:
df['CONSTRUCTION_START_DATE'] = pd.to_datetime(df['CONSTRUCTION_START_DATE'])
df['SUBSTANTIAL_COMPLETION_DATE'] = pd.to_datetime(df['SUBSTANTIAL_COMPLETION_DATE'])
construction_duration = df['SUBSTANTIAL_COMPLETION_DATE'] - df['CONSTRUCTION_START_DATE']

In [71]:
df.head()

,id,PROJECTNUMBER,MW,PROJECT_CITY,STATE,PROJECT_COUNTRY,CORE_MARKET,PROJECT_TYPE,SIZE_BUILDINGSIZE,NUMFLOORS,...,MasterItemNo,QtyShipped,UOM,ExtendedQuantity,PriceUOM,UnitPrice,ExtendedPrice,CONSTRUCTION_START_DATE,SUBSTANTIAL_COMPLETION_DATE,REVISED_ESTIMATE
0,3478,A0-0070,30.0,Mumbai,Maharashtra,India,Enterprise,Workspace,9000.0,0.0,...,58211,2.0,can,2.000,ML,1983.125,52605.000,2023-03-22,2023-05-25 12:00:00,2.409524e+07
1,8006,A0-0208,30.0,Delhi,Delhi,India,Wellness,Health Center,31200.0,4.0,...,53691,15.0,PC,0.150,ML,53273.000,7990.950,2022-11-01,2023-07-03 12:00:00,9.511661e+08
2,5748,A0-0004,30.0,Bengaluru,Karnataka,India,Enterprise,Workspace,2800000.0,0.0,...,63311,5.0,BG,5.000,BG,864.225,4321.125,2020-02-03,2025-10-31 00:00:00,3.492557e+10
3,7898,A0-0166,30.0,Chennai,Tamil Nadu,India,Tertiary Learning,Learning Hub,184000.0,0.0,...,61506,12.0,EA,12.000,EA,125.250,1503.000,2019-11-19,2025-01-24 12:00:00,1.553978e+10
4,11030,A0-0242,30.0,Hyderabad,Telangana,India,Wellness,Ambulatory Care,25000.0,4.0,...,50921,8.0,PC,0.128,ML,39245.000,23780.800,2021-12-03,2023-03-02 12:00:00,6.867939e+08


In [72]:
df.columns

Index(['id', 'PROJECTNUMBER', 'MW', 'PROJECT_CITY', 'STATE', 'PROJECT_COUNTRY',
       'CORE_MARKET', 'PROJECT_TYPE', 'SIZE_BUILDINGSIZE', 'NUMFLOORS',
       'NUMROOMS', 'NUMBEDS', 'invoiceId', 'invoiceDate', 'invoiceTotal',
       'ItemDescription', 'MasterItemNo', 'QtyShipped', 'UOM',
       'ExtendedQuantity', 'PriceUOM', 'UnitPrice', 'ExtendedPrice',
       'CONSTRUCTION_START_DATE', 'SUBSTANTIAL_COMPLETION_DATE',
       'REVISED_ESTIMATE'],
      dtype='object')

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
classifications = ["PROJECT_CITY", "STATE","CORE_MARKET", "PROJECT_TYPE","SIZE_BUILDINGSIZE", "NUMFLOORS", "NUMROOMS", "NUMBEDS", "invoiceTotal"]
regressions = ["MasterItemNo","UnitPrice", "ExtendedPrice", "PriceUOM","PROJECT_CITY","STATE", "PROJECT_TYPE","SIZE_BUILDINGSIZE", "NUMFLOORS", "REVISED_ESTIMATE"]
X_class = df[classifications].copy()
y_class = df["MasterItemNo"]
X_reg = df[regressions].copy()
y_reg = df["QtyShipped"]
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.1, random_state=42)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.1, random_state=42)

In [42]:
threshold = 50
counts = df["MasterItemNo"].value_counts()
rare_classes = counts[counts < threshold].index
df["MasterItemNo_grouped"] = df["MasterItemNo"].replace(rare_classes, "Other")

In [54]:
def preprocessor(X):
    numericals = X.select_dtypes(include=["int64","float64"]).columns
    categories = X.select_dtypes(include=["object", "category"]).columns

    return ColumnTransformer([
        ("num", StandardScaler(), numericals),
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), categories)])

In [55]:
reg_pipe = Pipeline([
    ("preprocessor", preprocessor(Xr_train)),
    ("model", RandomForestRegressor(n_estimators=150, random_state=42))])
reg_pipe.fit(Xr_train, yr_train)
preds = reg_pipe.predict(Xr_test)

RMSE (QtyShipped): 153.53647319795868


In [57]:
from sklearn.metrics import r2_score
r2_score(yr_test, preds)

0.9698670429419463

In [ ]:
classification = Pipeline([
    ("preprocessor", preprocessor(Xc_train)),
    ("model", RandomForestClassifier(n_estimators=150, random_state=42, class_weight="balanced"))
])
classification.fit(Xc_train, yc_train)
preds = classification.predict(Xc_test)